In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [11]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
    #                                                                 labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### repeatable!
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                 labels=tf.zeros_like(gQs3))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [12]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [13]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [14]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [15]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [16]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [17]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [18]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [19]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:1.3834 dloss:1.3884 aloss2:1.9898 exploreP:0.9980
Episode:1 meanR:19.5000 R:19.0000 rate:0.0380 aloss:1.3951 dloss:1.3862 aloss2:1.9702 exploreP:0.9961
Episode:2 meanR:30.0000 R:51.0000 rate:0.1020 aloss:1.3858 dloss:1.3530 aloss2:1.9983 exploreP:0.9911
Episode:3 meanR:30.0000 R:30.0000 rate:0.0600 aloss:1.3950 dloss:1.3385 aloss2:2.0000 exploreP:0.9882
Episode:4 meanR:29.4000 R:27.0000 rate:0.0540 aloss:1.3987 dloss:1.3135 aloss2:2.0156 exploreP:0.9856
Episode:5 meanR:26.6667 R:13.0000 rate:0.0260 aloss:1.4192 dloss:1.3721 aloss2:1.9181 exploreP:0.9843
Episode:6 meanR:28.8571 R:42.0000 rate:0.0840 aloss:1.4015 dloss:1.2940 aloss2:2.0305 exploreP:0.9802
Episode:7 meanR:31.7500 R:52.0000 rate:0.1040 aloss:1.3868 dloss:1.2584 aloss2:2.0565 exploreP:0.9752
Episode:8 meanR:30.0000 R:16.0000 rate:0.0320 aloss:1.3918 dloss:1.2571 aloss2:2.0445 exploreP:0.9736
Episode:9 meanR:28.2000 R:12.0000 rate:0.0240 aloss:1.3868 dloss:1.2442 aloss2:2.0

Episode:80 meanR:26.8272 R:49.0000 rate:0.0980 aloss:1.4075 dloss:0.8138 aloss2:3.2721 exploreP:0.8066
Episode:81 meanR:26.6585 R:13.0000 rate:0.0260 aloss:1.4177 dloss:0.7891 aloss2:3.4011 exploreP:0.8056
Episode:82 meanR:26.4940 R:13.0000 rate:0.0260 aloss:1.3813 dloss:0.7944 aloss2:3.4535 exploreP:0.8046
Episode:83 meanR:26.5714 R:33.0000 rate:0.0660 aloss:1.3986 dloss:0.8053 aloss2:3.3831 exploreP:0.8020
Episode:84 meanR:26.6706 R:35.0000 rate:0.0700 aloss:1.4064 dloss:0.7645 aloss2:3.5928 exploreP:0.7992
Episode:85 meanR:26.6744 R:27.0000 rate:0.0540 aloss:1.3883 dloss:0.7544 aloss2:3.7295 exploreP:0.7971
Episode:86 meanR:26.5977 R:20.0000 rate:0.0400 aloss:1.3993 dloss:0.7740 aloss2:3.5837 exploreP:0.7955
Episode:87 meanR:26.5227 R:20.0000 rate:0.0400 aloss:1.3829 dloss:0.7620 aloss2:3.7052 exploreP:0.7939
Episode:88 meanR:26.9551 R:65.0000 rate:0.1300 aloss:1.4007 dloss:0.7180 aloss2:3.9347 exploreP:0.7888
Episode:89 meanR:26.8667 R:19.0000 rate:0.0380 aloss:1.3964 dloss:0.7126 

Episode:159 meanR:28.0000 R:15.0000 rate:0.0300 aloss:1.3783 dloss:0.5103 aloss2:8.3488 exploreP:0.6527
Episode:160 meanR:27.5900 R:16.0000 rate:0.0320 aloss:1.3926 dloss:0.4632 aloss2:8.0631 exploreP:0.6517
Episode:161 meanR:27.5500 R:30.0000 rate:0.0600 aloss:1.3755 dloss:0.4823 aloss2:7.9771 exploreP:0.6498
Episode:162 meanR:27.6000 R:19.0000 rate:0.0380 aloss:1.3805 dloss:0.4640 aloss2:8.4877 exploreP:0.6486
Episode:163 meanR:27.4800 R:21.0000 rate:0.0420 aloss:1.3687 dloss:0.4647 aloss2:8.0572 exploreP:0.6472
Episode:164 meanR:27.3500 R:20.0000 rate:0.0400 aloss:1.3844 dloss:0.4501 aloss2:8.5884 exploreP:0.6459
Episode:165 meanR:27.7100 R:65.0000 rate:0.1300 aloss:1.3841 dloss:0.4733 aloss2:8.3425 exploreP:0.6418
Episode:166 meanR:27.7900 R:23.0000 rate:0.0460 aloss:1.3764 dloss:0.4778 aloss2:8.3204 exploreP:0.6404
Episode:167 meanR:27.0700 R:16.0000 rate:0.0320 aloss:1.3792 dloss:0.4844 aloss2:8.4134 exploreP:0.6394
Episode:168 meanR:27.5100 R:71.0000 rate:0.1420 aloss:1.3825 dlo

Episode:238 meanR:29.9700 R:137.0000 rate:0.2740 aloss:1.3812 dloss:0.4618 aloss2:10.0008 exploreP:0.5096
Episode:239 meanR:30.0100 R:35.0000 rate:0.0700 aloss:1.3812 dloss:0.4398 aloss2:9.9633 exploreP:0.5079
Episode:240 meanR:30.4300 R:97.0000 rate:0.1940 aloss:1.3747 dloss:0.4570 aloss2:9.9177 exploreP:0.5030
Episode:241 meanR:30.4000 R:9.0000 rate:0.0180 aloss:1.3782 dloss:0.4477 aloss2:9.9143 exploreP:0.5026
Episode:242 meanR:30.8000 R:52.0000 rate:0.1040 aloss:1.3710 dloss:0.4794 aloss2:9.7274 exploreP:0.5000
Episode:243 meanR:31.6600 R:106.0000 rate:0.2120 aloss:1.3913 dloss:0.4598 aloss2:9.3360 exploreP:0.4949
Episode:244 meanR:31.6800 R:28.0000 rate:0.0560 aloss:1.3851 dloss:0.4655 aloss2:9.0147 exploreP:0.4935
Episode:245 meanR:31.5100 R:19.0000 rate:0.0380 aloss:1.3981 dloss:0.4396 aloss2:9.0259 exploreP:0.4926
Episode:246 meanR:31.9600 R:80.0000 rate:0.1600 aloss:1.4042 dloss:0.4779 aloss2:8.7599 exploreP:0.4888
Episode:247 meanR:32.8100 R:101.0000 rate:0.2020 aloss:1.3979 

Episode:317 meanR:60.0300 R:110.0000 rate:0.2200 aloss:1.4296 dloss:0.5093 aloss2:8.1292 exploreP:0.3199
Episode:318 meanR:61.0600 R:125.0000 rate:0.2500 aloss:1.4285 dloss:0.5116 aloss2:8.1896 exploreP:0.3160
Episode:319 meanR:62.5200 R:158.0000 rate:0.3160 aloss:1.4087 dloss:0.5153 aloss2:8.1844 exploreP:0.3112
Episode:320 meanR:63.9700 R:188.0000 rate:0.3760 aloss:1.4035 dloss:0.5516 aloss2:7.6605 exploreP:0.3056
Episode:321 meanR:65.3500 R:156.0000 rate:0.3120 aloss:1.4332 dloss:0.5437 aloss2:7.6499 exploreP:0.3010
Episode:322 meanR:66.7400 R:149.0000 rate:0.2980 aloss:1.4769 dloss:0.5387 aloss2:7.8492 exploreP:0.2967
Episode:323 meanR:67.9300 R:140.0000 rate:0.2800 aloss:1.4772 dloss:0.5554 aloss2:7.9105 exploreP:0.2927
Episode:324 meanR:68.8700 R:142.0000 rate:0.2840 aloss:1.4598 dloss:0.5401 aloss2:8.0088 exploreP:0.2887
Episode:325 meanR:70.2100 R:160.0000 rate:0.3200 aloss:1.4515 dloss:0.5217 aloss2:8.1570 exploreP:0.2843
Episode:326 meanR:70.1500 R:157.0000 rate:0.3140 aloss:

...

Episode:1018 meanR:298.8300 R:311.0000 rate:0.6220 aloss:1.0383 dloss:0.8176 aloss2:8.4452 exploreP:0.0100
Episode:1019 meanR:297.6900 R:261.0000 rate:0.5220 aloss:1.0454 dloss:0.8188 aloss2:8.2966 exploreP:0.0100
Episode:1020 meanR:299.3900 R:429.0000 rate:0.8580 aloss:1.0383 dloss:0.7991 aloss2:8.3679 exploreP:0.0100
Episode:1021 meanR:297.5200 R:313.0000 rate:0.6260 aloss:1.0346 dloss:0.8055 aloss2:8.5796 exploreP:0.0100
Episode:1022 meanR:296.2100 R:227.0000 rate:0.4540 aloss:1.0361 dloss:0.8258 aloss2:8.5468 exploreP:0.0100
Episode:1023 meanR:296.4800 R:388.0000 rate:0.7760 aloss:1.0389 dloss:0.8013 aloss2:8.5180 exploreP:0.0100
Episode:1024 meanR:296.0900 R:334.0000 rate:0.6680 aloss:1.0371 dloss:0.7952 aloss2:8.6398 exploreP:0.0100
Episode:1025 meanR:295.0500 R:303.0000 rate:0.6060 aloss:1.0419 dloss:0.8075 aloss2:8.7721 exploreP:0.0100
Episode:1026 meanR:293.8400 R:332.0000 rate:0.6640 aloss:1.0383 dloss:0.7972 aloss2:8.7231 exploreP:0.0100
Episode:1027 meanR:293.6100 R:331.000

Episode:1095 meanR:283.6500 R:263.0000 rate:0.5260 aloss:0.9892 dloss:0.6919 aloss2:12.7267 exploreP:0.0100
Episode:1096 meanR:283.7700 R:290.0000 rate:0.5800 aloss:0.9870 dloss:0.6941 aloss2:13.1386 exploreP:0.0100
Episode:1097 meanR:283.4300 R:278.0000 rate:0.5560 aloss:0.9879 dloss:0.6852 aloss2:13.2346 exploreP:0.0100
Episode:1098 meanR:283.1500 R:243.0000 rate:0.4860 aloss:0.9940 dloss:0.6834 aloss2:12.8851 exploreP:0.0100
Episode:1099 meanR:283.2300 R:260.0000 rate:0.5200 aloss:0.9935 dloss:0.6967 aloss2:13.2351 exploreP:0.0100
Episode:1100 meanR:282.9300 R:294.0000 rate:0.5880 aloss:0.9910 dloss:0.6867 aloss2:13.3862 exploreP:0.0100
Episode:1101 meanR:282.1200 R:233.0000 rate:0.4660 aloss:0.9844 dloss:0.6789 aloss2:13.3452 exploreP:0.0100
Episode:1102 meanR:283.1700 R:351.0000 rate:0.7020 aloss:0.9941 dloss:0.6864 aloss2:13.2053 exploreP:0.0100
Episode:1103 meanR:282.7900 R:259.0000 rate:0.5180 aloss:0.9865 dloss:0.6894 aloss2:13.7782 exploreP:0.0100
Episode:1104 meanR:283.1700 

Episode:1171 meanR:265.7200 R:301.0000 rate:0.6020 aloss:1.0112 dloss:0.7062 aloss2:14.7308 exploreP:0.0100
Episode:1172 meanR:265.6900 R:251.0000 rate:0.5020 aloss:1.0119 dloss:0.7037 aloss2:14.4269 exploreP:0.0100
Episode:1173 meanR:264.8700 R:243.0000 rate:0.4860 aloss:1.0073 dloss:0.6914 aloss2:14.5254 exploreP:0.0100
Episode:1174 meanR:264.7400 R:248.0000 rate:0.4960 aloss:0.9960 dloss:0.7074 aloss2:14.8502 exploreP:0.0100
Episode:1175 meanR:264.8300 R:286.0000 rate:0.5720 aloss:1.0139 dloss:0.6993 aloss2:14.8058 exploreP:0.0100
Episode:1176 meanR:264.2200 R:242.0000 rate:0.4840 aloss:1.0000 dloss:0.7082 aloss2:14.3266 exploreP:0.0100
Episode:1177 meanR:263.3400 R:223.0000 rate:0.4460 aloss:1.0060 dloss:0.7156 aloss2:14.6126 exploreP:0.0100
Episode:1178 meanR:263.5000 R:287.0000 rate:0.5740 aloss:1.0041 dloss:0.7027 aloss2:15.0639 exploreP:0.0100
Episode:1179 meanR:263.0400 R:252.0000 rate:0.5040 aloss:1.0134 dloss:0.7030 aloss2:14.9349 exploreP:0.0100
Episode:1180 meanR:262.7000 

Episode:1247 meanR:281.4800 R:358.0000 rate:0.7160 aloss:1.0904 dloss:0.7511 aloss2:12.7133 exploreP:0.0100
Episode:1248 meanR:281.6100 R:242.0000 rate:0.4840 aloss:1.0939 dloss:0.7568 aloss2:12.4507 exploreP:0.0100
Episode:1249 meanR:281.4200 R:242.0000 rate:0.4840 aloss:1.0940 dloss:0.7430 aloss2:13.2297 exploreP:0.0100
Episode:1250 meanR:281.4800 R:261.0000 rate:0.5220 aloss:1.0937 dloss:0.7540 aloss2:12.8899 exploreP:0.0100
Episode:1251 meanR:281.0300 R:210.0000 rate:0.4200 aloss:1.0951 dloss:0.7571 aloss2:12.3261 exploreP:0.0100
Episode:1252 meanR:281.5300 R:315.0000 rate:0.6300 aloss:1.0957 dloss:0.7681 aloss2:12.2811 exploreP:0.0100
Episode:1253 meanR:282.6300 R:344.0000 rate:0.6880 aloss:1.0910 dloss:0.7534 aloss2:13.1894 exploreP:0.0100
Episode:1254 meanR:282.8400 R:252.0000 rate:0.5040 aloss:1.0903 dloss:0.7568 aloss2:13.0422 exploreP:0.0100
Episode:1255 meanR:284.0700 R:342.0000 rate:0.6840 aloss:1.0911 dloss:0.7591 aloss2:11.9466 exploreP:0.0100
Episode:1256 meanR:284.7200 

Episode:1323 meanR:318.1700 R:248.0000 rate:0.4960 aloss:0.9902 dloss:0.7023 aloss2:20.2591 exploreP:0.0100
Episode:1324 meanR:318.3600 R:347.0000 rate:0.6940 aloss:0.9939 dloss:0.7079 aloss2:20.3649 exploreP:0.0100
Episode:1325 meanR:318.2500 R:228.0000 rate:0.4560 aloss:0.9925 dloss:0.7198 aloss2:20.3846 exploreP:0.0100
Episode:1326 meanR:320.5600 R:500.0000 rate:1.0000 aloss:0.9915 dloss:0.7054 aloss2:20.3587 exploreP:0.0100
Episode:1327 meanR:319.5500 R:245.0000 rate:0.4900 aloss:0.9874 dloss:0.6996 aloss2:20.8630 exploreP:0.0100
Episode:1328 meanR:319.2100 R:361.0000 rate:0.7220 aloss:0.9906 dloss:0.7088 aloss2:21.3907 exploreP:0.0100
Episode:1329 meanR:317.7100 R:318.0000 rate:0.6360 aloss:0.9842 dloss:0.7006 aloss2:21.5295 exploreP:0.0100
Episode:1330 meanR:317.7100 R:283.0000 rate:0.5660 aloss:0.9840 dloss:0.6861 aloss2:21.5903 exploreP:0.0100
Episode:1331 meanR:317.2200 R:299.0000 rate:0.5980 aloss:0.9859 dloss:0.6920 aloss2:22.0618 exploreP:0.0100
Episode:1332 meanR:315.9500 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.